In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, max, avg, expr

spark = (SparkSession.builder.appName("MyDelta_App")
    .config("spark.executor.memory", "512m")
    .config("spark.driver.memory", "1g")
    .config('spark.ui.port', '4040')
    .getOrCreate()
    )
spark

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/07/25 16:56:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
df = spark.read.format("json").option("multiline","true").load("*.json")

In [3]:
def is_int(value):
    try:
        int(value)
        return True
    except ValueError:
        return False
new_col_names = ['q_'+col_name if is_int(col_name) else col_name for col_name in df.columns  ]

for col_name in df.columns:
    df = df.withColumnRenamed(col_name, "q_"+col_name if is_int(col_name) else col_name)

# df
array_columns = [col_name for col_name, data_type in df.dtypes if data_type.startswith("array") ]

In [4]:
from functools import reduce

collect_all_cols =  reduce(lambda acc, c : acc + expr(f"{c}[0]"), array_columns, expr("0.0"))
df.withColumn("total_time_seconds", collect_all_cols).select(
    'benchmark',
     'data_path',
     'engine',
     'query_path',
     'run_id',
     'total_time_seconds').show()

24/07/25 16:57:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+---------+------+------------+--------------------+------------------+
|benchmark|data_path|engine|  query_path|              run_id|total_time_seconds|
+---------+---------+------+------------+--------------------+------------------+
|     tpch|tpch/data|duckdb|tpch/queries|        duck_4cpu_2G|1.8507003784179688|
|     tpch|tpch/data|duckdb|tpch/queries|duck_1cpu_1g_0thr...| 9.805975914001465|
|     tpch|tpch/data|duckdb|tpch/queries|        duck_2cpu_1G| 5.028869390487671|
|     tpch|tpch/data|duckdb|tpch/queries|        duck_1cpu_1G|13.013150453567505|
|     tpch|tpch/data| spark|tpch/queries|spark_1core_1exec...|114.83095479011536|
|     tpch|tpch/data| spark|tpch/queries|spark_1core_1exec...|114.76612782478333|
|     tpch|tpch/data| spark|tpch/queries|   2core_2exec_512mb| 79.38333773612976|
+---------+---------+------+------------+--------------------+------------------+

